# Agent Function Calls
Run the DataOps agent entirely from Python—no Typer CLI invocation required.
This notebook shows how to instantiate `Settings`, build the agent, and call it directly.

## Setup
- Ensure you've installed dependencies via `uv pip install -e ".[dev]"`.
- Configure `.env` with your model/provider credentials.
- Optional: bring up the sample Postgres DB (`docker compose up -d postgres`) if you want SQL tooling.

In [ ]:
from pathlib import Path

def load_dotenv_if_available() -> None:
    """Load .env files from the project root so Settings sees secrets."""
    repo_root = Path.cwd()
    env_file = (repo_root / '.env')
    if not env_file.exists():
        print('No .env file found next to this notebook; skipping.')
        return
    try:
        from dotenv import load_dotenv  # type: ignore
    except ImportError:
        print('Install python-dotenv to auto-load .env files.')
        return
    load_dotenv(env_file)
    print(f'Loaded environment variables from {env_file}')

load_dotenv_if_available()

In [ ]:
from aiagent.config import get_settings

base_settings = get_settings()
# Toggle verbose tracing directly rather than exporting AGENT_VERBOSITY
settings = base_settings.model_copy(update={
    'verbose': True
})
settings.model_dump()


In [ ]:
from typing import Any, Dict, List
from aiagent.agent import build_agent
from langchain_core.messages import HumanMessage

agent = build_agent(settings)
conversation: Dict[str, Any] = {'messages': []}

def ask_agent(prompt: str) -> str:
    messages: List[Any] = list(conversation['messages'])
    messages.append(HumanMessage(content=prompt))
    response = agent.invoke({'messages': messages})
    if isinstance(response, dict) and isinstance(response.get('messages'), list):
        conversation['messages'] = response['messages']
    elif isinstance(response, list):
        conversation['messages'] = response
    else:
        conversation['messages'] = messages
    final_messages = conversation['messages']
    last = final_messages[-1] if final_messages else response
    text = getattr(last, 'content', None)
    if isinstance(text, list):
        chunks = [chunk.get('text') for chunk in text if isinstance(chunk, dict)]
        text = '\n'.join([c for c in chunks if isinstance(c, str)])
    if isinstance(text, str) and text.strip():
        return text.strip()
    if isinstance(response, dict) and isinstance(response.get('output'), str):
        return response['output']
    return str(response)


In [ ]:
for prompt in [
    'Summarize the purpose of this DataOps agent.',
    'What tools can you call right now?'
]:
    print(f'Prompt: {prompt}')
    print(ask_agent(prompt))
    print('-' * 40)


In [ ]:
from aiagent.tools import build_default_tools

tools = build_default_tools(settings)
tool_report = {tool.name: tool.description for tool in tools}
tool_report
